In [95]:
import tensorflow as tf

In [96]:
from tensorflow.keras.models import load_model

In [97]:
import pandas as pd
import numpy as np

In [98]:
import pickle

In [99]:
model = load_model('model.h5')

In [100]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,947 (11.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [101]:
with open('le.pkl', 'rb') as f:
    le = pickle.load(f)

In [102]:
with open('ohe.pkl', 'rb') as f:
    ohe = pickle.load(f)

In [103]:
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [104]:
data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [105]:
ohe.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [106]:
data['Geography']

'France'

In [107]:
geo_encoded = ohe.transform([[data['Geography']]])

C:\Users\HP\.conda\envs\onnx_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [108]:
geo_encoded

array([[1., 0., 0.]])

In [109]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out())

In [110]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [111]:
df = pd.DataFrame([data])

In [112]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [113]:
df = pd.concat([df.drop('Geography', axis='columns'), geo_encoded_df], axis='columns')

In [114]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [140]:
df['Gender'] = le.transform(df['Gender'])

In [142]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [148]:
df1 = scaler.transform(df)

In [154]:
df1 = pd.DataFrame(df1, columns=df.columns)

In [156]:
df1

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
df2 = scaler.transform(df)

In [160]:
df2

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [162]:
model.predict(df1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


array([[0.16517174]], dtype=float32)

In [164]:
model.predict(df1)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


0.16517174